# Тестовое задание Подолян

# **Задание 1.**

**Контекст**

Вы работаете над проектом [etoai.ru](http://etoai.ru/), который анализирует диалоги между клиентами и менеджерами и создает задачи в CRM для поддержания договоренностей с клиентами.

Недавно поступили жалобы от пользователей, что задачи формулируются неправильно. Проблема локализована на уровне LLM-модели: необходимо доработать используемый промт.


**Ваши задачи:**

1.	**Работа с данными:**

•	Предоставлен CSV-файл с диалогами и обратной связью от клиентов. (https://docs.google.com/spreadsheets/d/1kxoGhGT-e_jPaFJ4Po7giixzFzogLowDBDi_C2pbvME/edit?usp=sharing ) 

•	Загрузите данные, проведите их очистку и предобработку (например, удаление лишних символов, нормализация текста).

In [38]:
import pandas as pd
import re

In [53]:
file_path = 'C:test_dialogs_dataset - test_dialogs_dataset.csv'

In [54]:
try:
    df = pd.read_csv(file_path)
except:
    print("can't open the file")

In [55]:
pd.set_option('display.max_colwidth', None)
df.head()

,dialog_id,dialog_text,dialog_date
0,1,"[02.04.2024 19:23:00]Менеджер: Добрый день!! [02.04.2024 19:24:00]Клиент: Здравствуйте! Хотел бы узнать о новой акции... 😊🙂 [02.05.2024 10:23:00]Менеджер: Конечно, у нас есть отличное предложение!!!!---",02.04.2024
1,2,"[21.04.2024 19:23:00] Менеджер: Здравствуйте, чем могу помочь? [21.04.2024 19:33:00] Клиент: я хотел бы отменить свой заказ.... *** [21.04.2024 20:23:00] Менеджер: Хорошо, я помогу вам с этим!!!! 123",21.04.2024
2,3,*** [02.03.2024 20:23:00]Клиент: Я ищу информацию о вашем продукте... ?!! [02.04.2024 20:23:00]Менеджер: Конечно! Вот ссылка на наш сайт: https://www.google.com/search?q=%D1%85%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%D0%B8+%D0%BE%D0%BF%D1%82%D0%BE%D0%BC&sca_esv=7a56bf203bd23989&sxsrf=ADLYWIIFGAFGFYVK7L0CVtDKwsZe0FUj0A:1725536689619&ei=sZnZZrK7Jd3WwPAP4LKdkQc&start=70&sa=N&sstk=AagrsujloIGakXXzwPQTResGiqYUCDsiQQeHE8hqbZdLnjOnjQWBKBju1_pPz2kNX81keluH-6z_8dSYcdJ-5rqYmBlLWlyHELUTFw&ved=2ahUKEwjy17mL3auIAxVdKxAIHWBZJ3IQ8tMDegQILBAQ&biw=1756&bih=897&dpr=1.6,02.03.2024


Необходимо убрать таймстемпы, фиксирующие дату и время. Ещё ссылки. Они не нужны. Оставлю только буквы, цифры, основные знаки препинания.
Полную нормализацию (приведение к нижнему регистру, лемматизацию, удаление стоп-слов) считаю в данной задаче избыточной. Это может быть полезно для задач NLP, но если цель - извлечение задач для CRM, важно сохранить полную форму речи для точного понимания контеста без потери важных нюансов.

In [62]:
def clean_text(text):
    clean_text = re.sub(r'\[\d{2}.\d{2}.\d{4} \d{2}:\d{2}:\d{2}\]', ' ', text)
    clean_text = re.sub(r'http\S+', ' ', clean_text)
    clean_text = re.sub(r'[^а-яА-Яa-zA-Z0-9\s.,!?-]', ' ', clean_text)
    return clean_text.strip()

In [63]:
df['clean_dialog_text'] = df['dialog_text'].apply(clean_text)

In [64]:
df.head()

,dialog_id,dialog_text,dialog_date,clean_dialog_text
0,1,"[02.04.2024 19:23:00]Менеджер: Добрый день!! [02.04.2024 19:24:00]Клиент: Здравствуйте! Хотел бы узнать о новой акции... 😊🙂 [02.05.2024 10:23:00]Менеджер: Конечно, у нас есть отличное предложение!!!!---",02.04.2024,"Менеджер Добрый день!! Клиент Здравствуйте! Хотел бы узнать о новой акции... Менеджер Конечно, у нас есть отличное предложение!!!!---"
1,2,"[21.04.2024 19:23:00] Менеджер: Здравствуйте, чем могу помочь? [21.04.2024 19:33:00] Клиент: я хотел бы отменить свой заказ.... *** [21.04.2024 20:23:00] Менеджер: Хорошо, я помогу вам с этим!!!! 123",21.04.2024,"Менеджер Здравствуйте, чем могу помочь? Клиент я хотел бы отменить свой заказ.... Менеджер Хорошо, я помогу вам с этим!!!! 123"
2,3,*** [02.03.2024 20:23:00]Клиент: Я ищу информацию о вашем продукте... ?!! [02.04.2024 20:23:00]Менеджер: Конечно! Вот ссылка на наш сайт: https://www.google.com/search?q=%D1%85%D0%BE%D0%BB%D0%BE%D0%B4%D0%B8%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%D0%B8+%D0%BE%D0%BF%D1%82%D0%BE%D0%BC&sca_esv=7a56bf203bd23989&sxsrf=ADLYWIIFGAFGFYVK7L0CVtDKwsZe0FUj0A:1725536689619&ei=sZnZZrK7Jd3WwPAP4LKdkQc&start=70&sa=N&sstk=AagrsujloIGakXXzwPQTResGiqYUCDsiQQeHE8hqbZdLnjOnjQWBKBju1_pPz2kNX81keluH-6z_8dSYcdJ-5rqYmBlLWlyHELUTFw&ved=2ahUKEwjy17mL3auIAxVdKxAIHWBZJ3IQ8tMDegQILBAQ&biw=1756&bih=897&dpr=1.6,02.03.2024,Клиент Я ищу информацию о вашем продукте... ?!! Менеджер Конечно! Вот ссылка на наш сайт


In [59]:
df.to_csv('cleaned_df.csv', index=False)
print('Очищенный текст в cleaned_df.csv')

Очищенный текст в cleaned_df.csv


2. **Отладка промта**

•	Напишите новый промт для OpenAI модели, чтобы она создавала более релевантные задачи на основе диалогов.

•	Разработайте скрипт на Python, который:

•	Загружает и предобрабатывает датасет.

•	Отправляет диалоги с новым промтом в OpenAI API.

•	Выводит результаты.

**Важно!** 

В этом задании не ожидается от вас получить production ready Промт. Для нас важны три аспекта: 

1. умеете ли вы работать с данными 
2. ваши подходы и навыки к написанию промтом
3. *не забываете ли вы про задачу. 
Наша задача отладить промт, а значит в конце нам как-то нужно понять  “хорошо” он отработал или “плохо”.  Критерии и подходы можно написать в сопроводительной записке, это ок)*

In [ ]:
import openai

устанавливаем свой API-ключ OpenAI

In [ ]:
openai.api_key = 'api_key'

загружаю данные

In [ ]:
file_path1 = 'test_dialogs_dataset.csv'

In [ ]:
try:
    df = pd.read_csv(file_path1)
except:
    print("can't open the file")

напишу функцию предобработки данных, вроде тех, которые были в предыдущем задании

In [ ]:
def preprocess_text(text):
        # Удаление временных меток
        text = re.sub(r'\[\d{2}.\d{2}.\d{4} \d{2}:\d{2}:\d{2}\]', ' ', text)
        # Удаление ссылок
        text = re.sub(r'http\S+', ' ', text)
        # Удаление символов, не относящихся к тексту
        text = re.sub(r'[^а-яА-Яa-zA-Z0-9\s.,!?-]', ' ', text)
        #приведу к нижнему регистру, чтобы упростить обработку
        text = text.lower()
        return text.strip()


применяю предобработку к столбцу с текстом

In [ ]:
df['clean_dialog_text'] = df['dialog_text'].apply(preprocess_text)

Напишу функцию, которая создает задачу на основе диалога

In [ ]:
def create_task(dialog_text):
    prompt = f"""
    На основе следующего диалога между клиентом и менеджером, пожалуйста, создайте задачу в CRM-системе для 
    дальнейшего сопровождения клиента. Убедитесь, что задача чётко сформулирована и включает все необходимые 
    детали для выполнения. 

    Диалог:
    {dialog_text}

    Задача:
    """
     response = openai.Completion.create(
        engine="text-davinci-003",  # Или другой доступный движок
        prompt=prompt,
        max_tokens=150 #или другое допустимое количество токенов по желанию
    )
    return response.choices[0].text.strip()    

Применяю функцию ко всем строкам в столбце

In [ ]:
df['task'] = df['clean_dialog_text'].apply(create_task)
#при большом объему данных запросы могут занять много времени. В тестовых задачах можно ограничить количество обрабатываемых строк
#df['task'] = df['clean_dialog_text'].head(10).apply(create_task)

Чтобы посмотреть результат

In [ ]:
df[['dialog_id', 'task']].head()

Сохраняю результат в csv файл

In [ ]:
df.to_csv('tasks_output.csv', index=False)

#### Сопроводительная записка

1. Работа с данными
Для работы с данными был использован CSV-файл, содержащий диалоги между клиентами и менеджерами. Основные шаги:

- Загрузка данных: данные загружены с помощью библиотеки pandas в DataFrame с использованием функции pd.read_csv(). Для обеспечения устойчивости работы кода добавлена обработка ошибок.

- Очистка и предобработка данных: тексты диалогов прошли предобработку, в ходе которой:

Удалены временные метки.
Удалены ссылки.
Удалены лишние символы (смайлики, специальные символы).
Произведено удаление лишних пробелов, а также текст переведён к нижнему регистру, чтобы облегчить дальнейшую обработку.
Эти шаги позволяют преобразовать текст в формат, более удобный для дальнейшей работы с моделью, обеспечивая её правильное восприятие данных.

2. Написание промтов   

Создан промт для OpenAI модели, который описывает требуемую задачу в ясной и подробной форме. Промт включает вводную часть, описание контекста, предоставленный текст диалога и просьбу создать задачу в CRM-системе на основе данного диалога.
В промте модель просят создать задачу, содержащую четко сформулированные шаги, которые помогут в дальнейшем сопровождении клиента. Это повышает вероятность того, что результат будет содержать полезные и точные указания.

 3. Оценка качества работы промта
 
Для оценки качества можно использовать следующие критерии:

Соответствие задаче:

Полученные задачи должны быть логически связаны с содержанием диалога и соответствовать указанным в диалоге событиям.
Важные детали (например, запросы клиента, предлагаемые решения) должны быть включены в описание задачи.   


Структурированность задачи:

Задачи должны быть чёткими и конкретными, с указанием действий, которые следует предпринять для сопровождения клиента.
Будет хорошо, если  они включают сроки.   

Точность формулировки:

Задачи не должны быть слишком расплывчатыми или общими. Например, задача "позвонить клиенту" без контекста будет недостаточно релевантной, если в диалоге обсуждались конкретные действия или дальнейшие шаги.   


Исправление ошибок:

Если при анализе результатов обнаружены ошибки, например, несоответствие между диалогом и сформулированной задачей, это может сигнализировать о необходимости доработки промта. В таком случае нужно скорректировать его, добавив больше деталей или уточнений. Можно, например, добавить больше контекста или изменить формулировки

Удовлетворение бизнес-задаче:   

Задача должна включать шаги для удержания клиента, если это является целью. Хорошо, если модель генерирует задачи, направленные на достижение бизнес-целей и плохо, если задачи генерируются без учёта бизнес-логики.


*Как мы поймём, что промт отработал хорошо?*

- Можно оценить результат вручную. 
После того как модель сгенерировала задачу, стоит вручную проверить результат по перечисленным выше критериям. Если есть примеры правильных решений, можно сравнить результат работы модели с этими эталонами.

- Обратная связь от пользователей.
Можно собрать обратную связь от сотрудников, которые будут выполнять задачи, и от клиентов, если они участвуют в процессе.
Эти шаги помогут понять, насколько хорошо сработал промт и где могут потребоваться улучшения.

# **Задание 2.**

Представим, что вы с командой решили, что пора двигать продукт дальше.  Следующей итерацией вы решили сделать, чтобы ваш сервис не только ставил задачи на менеджеров в CRM, но и какие-то из задач закрывал сам. Например, если нужно клиенту напомнить о встрече — напомнил или если клиент “потерялся” и не ответил на сообщение менеджера, то пингануть клиента. 

Расскажите, пожалуйста, как вы считаете насколько реальна эта задача? 

Как бы вы подступились к реализации? 

**Важно!** 
В этом задании важны направления ваших рассуждений, их структурность  и обоснованность. Ничего кодить здесь не нужно)

Думаю, что автоматизация закрытия задач в CRM, таких как напоминание о встречах или пинг «потерявшихся» клиентов, вполне реальна и достижима с использованием современных технологий, таких как NLP и интеграцию с  CRM-системами. Скорее всего, ваша компания пользуется чем-то подобным.
Итак, у нас 2 задачи: напоминание о встрече (здесь нужна заранее согласованная информация о встрече) и пинг клиента через заданный период времени.
Для решения этих задач нам нужен доступ к SRM для автоматического создания, обновления и закрытия задач. Это включает интеграцию через API SRM-системы, вроде Salesforce, HubSpot, AmoSRM. Для работы системе нужен список клиентов и даты запланированных действийю   
Кроме того, нужны NLP модели (вроде GPT), когорые будут генерировать естественные и персонализированные сообщения для каждого клиента. 
Для автоматизации процессов необходимо настроить автоматические триггеры в SRM  или отдельно, чтобы система запускала напоминание или пинг. Например: напоминание о встрече за 3 часа до встречи и пинг через 5 дней после сообщения менеджера.
После генерации сообщения, сервис должен отправлять их автоматически, например месседжером.   
Задачу напоминания можно закрыть сразу после отправки сообщения, а задачу пинга может быть переведена в другой статус для дальнейшего усмотрения менеджера либо тоже закрываться после отправки сообщения.
Будет полезно организовать обратную связь от менеджеров, чтобы они отмечали правильность обработки задачи системой.